# Challenge:
The problem consists in predicting the probability of delay of the flights that land or take off from the airport of Santiago de Chile
(SCL). For that you will have a dataset using public and real data where each row corresponds to a flight that landed or took off
from SCL during 2017.


# Importar librerias y cargar el dataset

In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Cargar datos del .csv
original_data = pd.read_csv("dataset_SCL.csv",
                parse_dates=['Fecha-I', 'Fecha-O'],
                dtype = {                
                'Vlo-I': 'string',
                'Ori-I' : 'string' ,
                'Des-I' : 'string' ,
                'Emp-I': 'string',
                'Vlo-O': 'string',
                'Ori-O': 'string', 
                'Des-O': 'string',
                'Emp-O': 'string', 
                'DIA': 'int64', 
                'MES': 'int64', 
                'AÑO': 'int64', 
                'DIANOM': 'string', 
                'TIPOVUELO': 'string',
                'OPERA': 'string', 
                'SIGLAORI': 'string', 
                'SIGLADES': 'string'
                }
                )
original_data.head(5)

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,1,1,2017,Domingo,I,American Airlines,Santiago,Miami
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,2,1,2017,Lunes,I,American Airlines,Santiago,Miami
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,3,1,2017,Martes,I,American Airlines,Santiago,Miami
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,4,1,2017,Miercoles,I,American Airlines,Santiago,Miami
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,5,1,2017,Jueves,I,American Airlines,Santiago,Miami


- Fecha-I: Scheduled date and time of the flight.
- Vlo-I : Scheduled flight number.
- Ori-I : Programmed origin city code.
- Des-I : Programmed destination city code.
- Emp-I : Scheduled flight airline code.
- Fecha-O : Date and time of flight operation.
- Vlo-O : Flight operation number of the flight.
- Ori-O : Operation origin city code
- Des-O : Operation destination city code.
- Emp-O : Airline code of the operated flight.
- DIA: Day of the month of flight operation.
- MES : Number of the month of operation of the flight.
- AÑO : Year of flight operation.
- DIANOM : Day of the week of flight operation.
- TIPOVUELO : Type of flight, I =International, N =National.
- OPERA : Name of the airline that operates.
- SIGLAORI: Name city of origin.
- SIGLADES: Destination city name.

# Procesamiento de datos: Verificación de nulos y consistencia en el formato de variables

In [3]:
#Cantidad total de filas
print(f"Cantidad de filas totales: {len(original_data)}")

#Revisar cantidad de nulos por columna
print(f"Cantidad de valores nulos por columna: {original_data.isna().sum()}")

#Existe solo 1 fila con un valor nulo, debido a que es solo una no hay problema si se elimina

not_null_data = original_data.dropna().copy()

print(f"Cantidad de filas totales del dataframe filtrado: {len(not_null_data)}")
print(f"Cantidad de filas nulas en dataframe filtrado: {not_null_data.isna().sum().sum()}")

Cantidad de filas totales: 68206
Cantidad de valores nulos por columna: Fecha-I      0
Vlo-I        0
Ori-I        0
Des-I        0
Emp-I        0
Fecha-O      0
Vlo-O        1
Ori-O        0
Des-O        0
Emp-O        0
DIA          0
MES          0
AÑO          0
DIANOM       0
TIPOVUELO    0
OPERA        0
SIGLAORI     0
SIGLADES     0
dtype: int64
Cantidad de filas totales del dataframe filtrado: 68205
Cantidad de filas nulas en dataframe filtrado: 0


## Valores unicos por columna:
- Se puede observar que la tanto Vlo-I, DES-I y EMP-I no tienen la misma cantidad de valores unicos que sus contrapartes de operación, hay que revisar si esto se debe a irregularidades de formato en dichas variables

In [4]:
#Cantidad de valores unicos por columna 
print("Cantidad de valores unicos por columna")
not_null_data.nunique(axis=0)

Cantidad de valores unicos por columna


Fecha-I      53252
Vlo-I          584
Ori-I            1
Des-I           64
Emp-I           30
Fecha-O      62774
Vlo-O          861
Ori-O            1
Des-O           63
Emp-O           32
DIA             31
MES             12
AÑO              2
DIANOM           7
TIPOVUELO        2
OPERA           23
SIGLAORI         1
SIGLADES        62
dtype: int64

## Código de vuelo: 
- Existen 584 vuelos únicos en el campo de vuelos programados, mientras que en vuelos operados tenemos 861 valores únicos.
- Mirando algunos casos nos podemos dar cuenta que hay ID de vuelos en formato "float", como por ejemplo '1.0' o '10.0'. Asumiré que estos vuelos son lo mismo que 1 o 10, aplicaré una función para normalizar dichos valores en enteros.

In [5]:
#Vuelos programados unicos
print(f"ID unicos de los vuelos programados:  {not_null_data['Vlo-I'].sort_values().values.unique()}")

#Vuelos operados unicos
print(f"ID unicos de los vuelos realizados: {not_null_data['Vlo-O'].sort_values().values.unique()}")


ID unicos de los vuelos programados:  <StringArray>
[   '1',   '10',  '100', '1003', '1004', '1005', '1011',  '102', '1025',
 '1031',
 ...
   '98', '9851',  '986',  '988', '989P',   '99',  '991',  '993', '9955',
 '9956']
Length: 584, dtype: string
ID unicos de los vuelos realizados: <StringArray>
[   '1',  '1.0',   '10', '10.0',  '100', '1001', '1003', '1004', '1005',
 '1011',
 ...
  '986', '9860',  '988',  '989',   '99', '99.0',  '991',  '993', '9955',
 '9956']
Length: 861, dtype: string


In [6]:
#Transformación de valores ingresados como float a int.
not_null_data.loc[:,not_null_data.columns=='Vlo-I'] = not_null_data['Vlo-I'].str.split('.').str[0].values
not_null_data.loc[:,not_null_data.columns=='Vlo-O'] = not_null_data['Vlo-O'].str.split('.').str[0].values

print(f'''Valores únicos en Vlo-I: {not_null_data['Vlo-I'].nunique()}''')
print(f'''Valores únicos en Vlo-O: {not_null_data['Vlo-O'].nunique()}''')
print("\n Para el caso de Vlo-O habian 253 casos de valores enteros ingresados como floats, mientras que en Vlo-I ninguno.")

Valores únicos en Vlo-I: 584
Valores únicos en Vlo-O: 608

 Para el caso de Vlo-O habian 253 casos de valores enteros ingresados como floats, mientras que en Vlo-I ninguno.


C:\Users\crist\AppData\Local\Temp\ipykernel_10848\1949629676.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  not_null_data.loc[:,not_null_data.columns=='Vlo-I'] = not_null_data['Vlo-I'].str.split('.').str[0].values
C:\Users\crist\AppData\Local\Temp\ipykernel_10848\1949629676.py:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  not_null_data.loc[:,not_null_data.columns=='Vlo-O'] = not_null_data['Vlo-O'].str.split('.').str[0].values


## Código de destino

- Existen 64 destinos únicos programados mientras que en operacion hay 63, no se observa algún problema ya que pareciera estar todo en el mismo formato.

In [7]:
print(f"ID unicos de destinos programados:  {not_null_data['Des-I'].sort_values().values.unique()}")

print(f"ID unicos de destinos realizados: {not_null_data['Des-O'].sort_values().values.unique()}")

ID unicos de destinos programados:  <StringArray>
['CYYZ', 'EGLL', 'EGYP', 'KATL', 'KDFW', 'KIAH', 'KJFK', 'KLAX', 'KMCO',
 'KMIA', 'LEMD', 'LFPG', 'LIRF', 'MDPC', 'MMMX', 'MMUN', 'MPTO', 'NZAA',
 'SAAR', 'SABE', 'SACO', 'SAEZ', 'SAME', 'SANT', 'SANU', 'SARI', 'SAWH',
 'SAZN', 'SAZS', 'SBCT', 'SBFI', 'SBFL', 'SBGL', 'SBGR', 'SCAR', 'SCAT',
 'SCBA', 'SCCF', 'SCCI', 'SCDA', 'SCFA', 'SCIE', 'SCIP', 'SCJO', 'SCNT',
 'SCPQ', 'SCQP', 'SCSE', 'SCTE', 'SCVD', 'SEGU', 'SEQM', 'SEQU', 'SGAS',
 'SKBO', 'SLCB', 'SLLP', 'SLVR', 'SPJC', 'SPSO', 'SULS', 'SUMU', 'YMML',
 'YSSY']
Length: 64, dtype: string
ID unicos de destinos realizados: <StringArray>
['CYYZ', 'EGLL', 'EGYP', 'KATL', 'KDFW', 'KIAD', 'KIAH', 'KJFK', 'KLAX',
 'KMCO', 'KMIA', 'LEMD', 'LFPG', 'LIRF', 'MDPC', 'MMMX', 'MMUN', 'MPTO',
 'NZAA', 'SAAR', 'SABE', 'SACO', 'SAEZ', 'SAME', 'SANT', 'SANU', 'SAWH',
 'SAZN', 'SAZS', 'SBCT', 'SBFI', 'SBFL', 'SBGL', 'SBGR', 'SCAR', 'SCAT',
 'SCBA', 'SCCF', 'SCCI', 'SCDA', 'SCFA', 'SCIE', 'SCIP', 'SCJO',

## Código de aerolinea:
- Existen 30 aerolineas distintas en el campo de programadas mientras que en las de operacion 32, revisando nos damos cuenta que efectivamente hay aerolineas que no estan en ambas listas, de todas formas siguen el mismo formato de 3 caracteres por lo que asumire que esto está bien.

In [8]:
print(f"ID unicos de los vuelos programados:  {not_null_data['Emp-I'].sort_values().values.unique()}")

print(f"ID unicos de los vuelos realizados: {not_null_data['Emp-O'].sort_values().values.unique()}")

ID unicos de los vuelos programados:  <StringArray>
['AAL', 'ACA', 'AFR', 'AMX', 'ARG', 'AUT', 'AVA', 'AZA', 'BAW', 'CMP', 'DAL',
 'DSM', 'GLO', 'IBE', 'JAT', 'JMR', 'KLM', 'LAN', 'LAP', 'LAW', 'LNE', 'LPE',
 'LRC', 'LXP', 'ONE', 'PUE', 'QFU', 'SKU', 'TAM', 'UAL']
Length: 30, dtype: string
ID unicos de los vuelos realizados: <StringArray>
['48O', '56R', 'AAL', 'ACA', 'AFR', 'AMX', 'ARG', 'AUT', 'AVA', 'AZA', 'BAW',
 'CMP', 'DAL', 'DSM', 'GLO', 'IBE', 'JAT', 'JMR', 'KLM', 'LAN', 'LAP', 'LNE',
 'LPE', 'LRC', 'LXP', 'ONE', 'PUE', 'QFA', 'SKU', 'TAM', 'TPU', 'UAL']
Length: 32, dtype: string


## Revisión del resto de variables: Observando los valores únicos en cada variable no se encontró ninguna irregularidad.
- DIA: ok
- MES: ok
- AÑO: ok
- DIANOM: ok
- TIPOVUELO: ok
- OPERA: ok
- SIGLAORI: ok, todos los vuelos salen de Santiago
- SIGLADES: ok

In [9]:
print(f"""\nValores únicos para día: \n {not_null_data["DIA"].sort_values().unique()}""")
print(f"""\nValores únicos para mes: \n {not_null_data["MES"].sort_values().unique()}""")
print(f"""\nValores únicos para año: \n {not_null_data["AÑO"].sort_values().unique()}""")
print(f"""\nValores únicos para día nominal: \n {not_null_data["DIANOM"].sort_values().unique()}""")
print(f"""\nValores únicos para tipo vuelo: \n {not_null_data["TIPOVUELO"].sort_values().unique()}""")
print(f"""\nValores únicos aerolinea: \n {not_null_data["OPERA"].sort_values().unique()}""")
print(f"""\nValores únicos origen: \n {not_null_data["SIGLAORI"].sort_values().unique()}""")
print(f"""\nValores únicos destino: \n {not_null_data["SIGLADES"].sort_values().unique()}""")


Valores únicos para día: 
 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31]

Valores únicos para mes: 
 [ 1  2  3  4  5  6  7  8  9 10 11 12]

Valores únicos para año: 
 [2017 2018]

Valores únicos para día nominal: 
 <StringArray>
['Domingo', 'Jueves', 'Lunes', 'Martes', 'Miercoles', 'Sabado', 'Viernes']
Length: 7, dtype: string

Valores únicos para tipo vuelo: 
 <StringArray>
['I', 'N']
Length: 2, dtype: string

Valores únicos aerolinea: 
 <StringArray>
[   'Aerolineas Argentinas',               'Aeromexico',
               'Air Canada',               'Air France',
                 'Alitalia',        'American Airlines',
                  'Austral',                  'Avianca',
          'British Airways',                 'Copa Air',
                'Delta Air',                'Gol Trans',
              'Grupo LATAM',                   'Iberia',
             'JetSmart SPA',                   'K.L.M.',
                    'Lacsa',     'La

# Creación de variables solicitadas:
- high_season : 1 if Date-I is between Dec-15 and Mar-3, or Jul-15 and Jul-31, or Sep-11 and Sep-30, 0 otherwise.
- min_diff : difference in minutes between Date-O and Date-I .
- delay_15 : 1 if min_diff > 15, 0 if not.
- period_day : morning (between 5:00 and 11:59), afternoon (between 12:00 and 18:59) and night (between 19:00 and 4:59), based onDate-I.


In [10]:
# Una vez revisada la integridad de los datos se crea un nuevo dataframe con los datos limpios.
clean_data = not_null_data.copy()